In [ ]:
#Add Python module called Mediapipe
!pip install mediapipe

In [ ]:
import tensorflow as tf


In [ ]:
# Import the much needed stuff for training
import pandas as pd
import numpy as np
import tensorflow as tf
import mediapipe as mp
import os
import csv
import cv2 as cv
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from sklearn.metrics import confusion_matrix

from keras.utils import to_categorical
import seaborn as sns
# Make numpy values easier to read.
np.set_printoptions(precision=3, suppress=True)

# Checking Tensorflow Version
tf.__version__

In [ ]:
# Function to Extract Feature from images or Frame
def extract_feature(input_image):
    mp_hands = mp.solutions.hands
    mp_drawing = mp.solutions.drawing_utils 
    image = cv.imread(input_image)
    
    with mp_hands.Hands(static_image_mode=True, max_num_hands=2, min_detection_confidence=0.1) as hands:
        while True:
            results = hands.process(cv.flip(cv.cvtColor(image, cv.COLOR_BGR2RGB), 1))
            image_height, image_width, _ = image.shape
            # Print handedness (left v.s. right hand).
            # Caution : Uncomment these print command will resulting long log of mediapipe log
            #print(f'Handedness of {input_image}:')
            #print(results.multi_handedness)

            # Draw hand landmarks of each hand.
            # Caution : Uncomment these print command will resulting long log of mediapipe log
            #print(f'Hand landmarks of {input_image}:')
            if not results.multi_hand_landmarks:
                # Here we will set whole landmarks into zero as no handpose detected
                # in a picture wanted to extract.
                
                # Wrist Hand
                hand_data=np.zeros(10)
                
                # Set image to Zero
                annotated_image = 0

                # Return Whole Landmark and Image
                return (hand_data,
                        annotated_image)
            
            annotated_image = cv.flip(image.copy(), 1)
            hand_data = []

            # Iterate over all hands detected in the image.
            for hand_landmarks in results.multi_hand_landmarks:

                # Extract the coordinates of all 21 hand landmarks.
                landmark_coordinates = []
                for landmark in mp_hands.HandLandmark:
                    landmark_coordinates.append(hand_landmarks.landmark[landmark].x * image_width)
                    landmark_coordinates.append(hand_landmarks.landmark[landmark].y * image_height)
                    landmark_coordinates.append(hand_landmarks.landmark[landmark].z)
                mp_drawing.draw_landmarks(annotated_image, hand_landmarks, mp_hands.HAND_CONNECTIONS)

                # Add the coordinates of all 21 hand landmarks for the current hand to the new array.
                hand_data.extend(landmark_coordinates)
            #If less than two hands are detected, fill the rest of the array with zeros.
            if len(results.multi_hand_landmarks) < 2:
                num_missing_hands = 2 - len(results.multi_hand_landmarks)
                num_missing_coordinates = num_missing_hands * 21 * 3  # 21 landmarks * 3 coordinates (x, y, z)
                hand_data.extend([0] * num_missing_coordinates)

            hand_data = np.array(hand_data)
            return  (hand_data, annotated_image)

In [ ]:
import csv
import os

def toCSV(filecsv, class_type, hand_data):
    # Convert elements of hand_data to string
    hand_data = list(map(str, hand_data))
    
    # Check if file exists
    if os.path.isfile(filecsv):
        with open(filecsv, 'a+', newline='') as file:
            writer = csv.writer(file)
            # Write the class type and hand data to the file
            writer.writerow([class_type] + hand_data)
    else:
        with open(filecsv, 'w', newline='') as file:
            writer = csv.writer(file)
            # Write the header to the file
            header = ["class_type"] + ["value"+str(i) for i in range(1, 127)]
            writer.writerow(header)
            # Write the class type and hand data to the file
            writer.writerow([class_type] + hand_data)


In [ ]:
# Your existing code...
paths = "/kaggle/input/tr-sign-language/tr_signLanguage_dataset/train/"
csv_path = "hands_SIBI_training.csv"

if os.path.exists(csv_path):
    print("CSV File does exist, going delete before start extraction and replace it with new")
    os.remove(csv_path)
else:
    print("The CSV file does not exist", csv_path, ",Going Create after Extraction")
    
for dirlist in os.listdir(paths):
    for root, directories, filenames in os.walk(os.path.join(paths, dirlist)):
        print("Inside Folder", dirlist, "Consist :", len(filenames), "Imageset")
        for filename in filenames:
            if filename.endswith(".jpg") or filename.endswith(".JPG"):
                # Assuming extract_feature returns an array of 126 values
                (hand_data, output_IMG) =  extract_feature(os.path.join(root, filename))
                if len(hand_data) == 126:
                    toCSV(csv_path, dirlist, hand_data)
                else :
                    print(os.path.join(root, filename), "Hand does not have landmarks")

print("===================Feature Extraction for TRAINING is Completed===================")

In [ ]:
# Your existing code...
paths = "/kaggle/input/tr-sign-language/tr_signLanguage_dataset/test/"
csv_path = "hands_SIBI_validation.csv"

if os.path.exists(csv_path):
    print("CSV File does exist, going delete before start extraction and replace it with new")
    os.remove(csv_path)
else:
    print("The CSV file does not exist", csv_path, ",Going Create after Extraction")
    
for dirlist in os.listdir(paths):
    for root, directories, filenames in os.walk(os.path.join(paths, dirlist)):
        print("Inside Folder", dirlist, "Consist :", len(filenames), "Imageset")
        for filename in filenames:
            if filename.endswith(".jpg") or filename.endswith(".JPG"):
                # Assuming extract_feature returns an array of 126 values
                (hand_data, output_IMG) =  extract_feature(os.path.join(root, filename))
                if len(hand_data) == 126:
                    toCSV(csv_path, dirlist, hand_data)
                else :
                    print(os.path.join(root, filename), "Hand does not have landmarks")

print("===================Feature Extraction for Testing is Completed===================")

In [ ]:
# Read CSV file for Training the model using Pandas
df_train = pd.read_csv("hands_SIBI_training.csv", header=0)

# First we must sort the values of the dataset according to the Alphabets
df_train = df_train.sort_values(by=["class_type"])

df_train.head()

In [ ]:
# class_counts = df_train['class_type'].value_counts()
# class_counts

In [ ]:
len(df_train['class_type'].unique())

In [ ]:
df_train['class_type'].unique()

In [ ]:
# lst = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M',
#        'N', 'O', 'P', 'R', 'S', 'T', 'U', 'V', 'Y', 'Z', 'del', 'nothing',
#        'space']

# dict_lst = {k: v for v, k in enumerate(lst)}

# print(dict_lst)


In [ ]:
# Read CSV file for Validation or Testing the Model using Pandas
df_test = pd.read_csv("hands_SIBI_validation.csv", header=0)

# First we must sort the values of the dataset according to the Alphabets
df_test = df_test.sort_values(by=["class_type"])

df_test

In [ ]:
# Put Categorical using Pandas
df_train["class_type"] = pd.Categorical(df_train["class_type"])
df_train["class_type"] = df_train.class_type.cat.codes

df_test["class_type"] = pd.Categorical(df_test["class_type"])
df_test["class_type"] = df_test.class_type.cat.codes

In [ ]:
X = df_train.drop('class_type', axis=1)
y=df_train["class_type"]

x_test = df_test.drop('class_type', axis=1)
y_test=df_test["class_type"]

In [ ]:
from sklearn.model_selection import train_test_split

# Assuming X is your data and y are your labels
x_train, x_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:

x_train = np.array(x_train)
x_val = np.array(x_val)
x_test = np.array(x_test)


In [ ]:
# Check Array Shape before transformation
print(x_train.shape)
print(x_test.shape)
print(x_val.shape)

# Since the array shape is 1x1, we must turn it into 1x10x1 so we can feed it into the model
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))
x_val = np.reshape(x_val, (x_val.shape[0], x_val.shape[1], 1))

# Check Array Shape after transformation
print(x_train.shape)
print(x_test.shape)
print(x_val.shape)


In [ ]:
# Check sample train and test features
print(x_train[0])
print(x_test[7])

In [ ]:
# Number of classes according standard Indonesian Language Alphabets
num_classes = 26

# Using the Keras.Utils to put the label categorically 
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)
y_val = to_categorical(y_val, num_classes)

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

# Name of the model
model_name = "best_model.h5"

# ModelCheckpoint callback
checkpoint = ModelCheckpoint(model_name,
                             monitor="val_loss",
                             mode="min",
                             save_best_only=True,
                             verbose=1)

learning_rate_reduction = ReduceLROnPlateau(monitor='val_loss',
                                            patience=2,  # Reduced patience to 2
                                            verbose=1,
                                            factor=0.3,  # Changed factor to 0.3
                                            min_lr=0.00001)  # Added minimum learning rate

In [ ]:
from tensorflow.keras import regularizers
from tensorflow.keras.optimizers import Adam

# One Dimensional Convolutional Neural Network model, Train will be feed to 1 Dimension Convolutional Neural Network
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv1D(filters=32, kernel_size=5, strides=1, padding="causal", activation="relu", input_shape=x_train.shape[1:3]),
    tf.keras.layers.Conv1D(filters=32, kernel_size=5, strides=1, padding="causal", activation="relu"),
    tf.keras.layers.MaxPooling1D(pool_size=2),
    tf.keras.layers.Conv1D(filters=64, kernel_size=5, strides=1, padding="causal", activation="relu"),
    tf.keras.layers.Conv1D(filters=64, kernel_size=5, strides=1, padding="causal", activation="relu"),
    tf.keras.layers.MaxPooling1D(pool_size=2),
    tf.keras.layers.Conv1D(filters=128, kernel_size=5, strides=1, padding="causal", activation="relu"),
    tf.keras.layers.Conv1D(filters=128, kernel_size=5, strides=1, padding="causal", activation="relu"),
    tf.keras.layers.MaxPooling1D(pool_size=2),
    tf.keras.layers.Conv1D(filters=256, kernel_size=5, strides=1, padding="causal", activation="relu"),
    tf.keras.layers.Conv1D(filters=256, kernel_size=5, strides=1, padding="causal", activation="relu"),
    tf.keras.layers.MaxPooling1D(pool_size=2),
    tf.keras.layers.Dropout(rate=0.2),
    # Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(),
    # 512 neuron hidden layer
    tf.keras.layers.Dense(512, activation='relu'), 
    tf.keras.layers.Dense(num_classes, activation='softmax')])

model.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()


In [ ]:
#Train the Model
history=model.fit(x_train, y_train, epochs=50, batch_size=32, validation_data=(x_val, y_val),callbacks=[checkpoint,learning_rate_reduction])

In [ ]:
# # One Dimensional Convolutional Neural Network model, Train will be feed to 1 Dimension Convolutional Neural Network
# model = tf.keras.models.Sequential([
#     tf.keras.layers.Conv1D(filters=32, kernel_size=5, strides=1, padding="causal", activation="relu", input_shape=x_train.shape[1:3]),
#     tf.keras.layers.Conv1D(filters=32, kernel_size=5, strides=1, padding="causal", activation="relu"),
#     tf.keras.layers.MaxPooling1D(pool_size=2),
#     tf.keras.layers.Conv1D(filters=64, kernel_size=5, strides=1, padding="causal", activation="relu"),
#     tf.keras.layers.Conv1D(filters=64, kernel_size=5, strides=1, padding="causal", activation="relu"),
#     tf.keras.layers.MaxPooling1D(pool_size=2),
#     tf.keras.layers.Conv1D(filters=128, kernel_size=5, strides=1, padding="causal", activation="relu"),
#     tf.keras.layers.Conv1D(filters=128, kernel_size=5, strides=1, padding="causal", activation="relu"),
#     tf.keras.layers.MaxPooling1D(pool_size=2),
#     tf.keras.layers.Conv1D(filters=256, kernel_size=5, strides=1, padding="causal", activation="relu"),
#     tf.keras.layers.Conv1D(filters=256, kernel_size=5, strides=1, padding="causal", activation="relu"),
#     tf.keras.layers.MaxPooling1D(pool_size=2),
#     tf.keras.layers.Dropout(rate=0.2),
#     # Flatten the results to feed into a DNN
#     tf.keras.layers.Flatten(),
#     # 512 neuron hidden layer
#     tf.keras.layers.Dense(512, activation='relu'), 
#     tf.keras.layers.Dense(num_classes, activation='softmax')])

# model.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# model.summary()

In [ ]:
plt.figure(figsize=(20,8))
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
plt.figure(figsize=(20,8))
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
#Saving the model into H5 system file
save_model = "model_tur.h5"
model.save(save_model)
print("Model Saved into", save_model)

In [ ]:
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=2)

print('\nTest accuracy:', test_acc)
print('\nTest loss:', test_loss)

In [ ]:
y_test = np.argmax(y_test,axis=1)
y_pred = model.predict(x_test)
y_pred = np.argmax(y_pred,axis=1)

In [ ]:
plt.figure(figsize=(10,10))
sns.heatmap(confusion_matrix(y_test,y_pred),annot=True,fmt='.3g',cmap='viridis')
plt.show()

In [ ]:
import tensorflow as tf
# Convert the model.
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the model.
with open('model_tur.tflite', 'wb') as f:
  f.write(tflite_model)

In [ ]:
# Hard Encode for the Prediction
classes = {'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5, 'G': 6, 'H': 7, 'I': 8, 'J': 9, 'K': 10, 'L': 11, 'M': 12, 'N': 13, 'O': 14, 'P': 15, 'R': 16, 'S': 17, 'T': 18, 'U': 19, 'V': 20, 'Y': 21, 'Z': 22, 'del': 23, 'nothing': 24, 'space': 25}

In [ ]:
import cv2

In [ ]:
# Directly from Imageset Dataset Testing
#Load Image and do Feature Extraction
path_to_image = "/kaggle/input/tr-sign-language/tr_signLanguage_dataset/test/U/U_0_100.jpg"

# (wristX, wristY, wristZ,
#  thumb_CmcX, thumb_CmcY, thumb_CmcZ,
#  thumb_McpX, thumb_McpY, thumb_McpZ,
#  thumb_IpX, thumb_IpY, thumb_IpZ,
#  thumb_TipX, thumb_TipY, thumb_TipZ,
#  index_McpX, index_McpY, index_McpZ,
#  index_PipX, index_PipY, index_PipZ,
#  index_DipX, index_DipY, index_DipZ,
#  index_TipX, index_TipY, index_TipZ,
#  middle_McpX, middle_McpY, middle_McpZ,
#  middle_PipX, middle_PipY, middle_PipZ,
#  middle_DipX, middle_DipY, middle_DipZ,
#  middle_TipX, middle_TipY, middle_TipZ,
#  ring_McpX, ring_McpY, ring_McpZ,
#  ring_PipX, ring_PipY, ring_PipZ,
#  ring_DipX, ring_DipY, ring_DipZ,
#  ring_TipX, ring_TipY, ring_TipZ,
#  pinky_McpX, pinky_McpY, pinky_McpZ,
#  pinky_PipX, pinky_PipY, pinky_PipZ,
#  pinky_DipX, pinky_DipY, pinky_DipZ,
#  pinky_TipX, pinky_TipY, pinky_TipZ,
(input_IMG, output_IMG) = extract_feature(path_to_image)

#print(wristX, wristY,
#      thumb_CmcX, thumb_CmcY, thumb_McpX, thumb_McpY, thumb_IpX, thumb_IpY, thumb_TipX, thumb_TipY,
#      index_McpX, index_McpY, index_PipX, index_PipY, index_DipX, index_DipY, index_TipX, index_TipY,
#      middle_McpX, middle_McpY, middle_PipX, middle_PipY, middle_DipX, middle_DipY, middle_TipX, middle_TipY,
#      ring_McpX, ring_McpY, ring_PipX, ring_PipY, ring_DipX, ring_DipY, ring_TipX, ring_TipY,
#      pinky_McpX, pinky_McpY, pinky_PipX, pinky_PipY, pinky_DipX, pinky_DipY, pinky_TipX, pinky_TipY)
plt.axis("on")
plt.imshow(cv.cvtColor(output_IMG, cv.COLOR_BGR2RGB))
plt.show()

In [ ]:
input_IMG=input_IMG.reshape(-1,1)
input_IMG = np.expand_dims(input_IMG, axis=0)
input_IMG.shape

In [ ]:
#Print the Prediction
print(model.predict(input_IMG))

In [ ]:
#Print prediction using defined Classes
predictions = model.predict(input_IMG)
predictions=np.argmax(predictions,axis=1)
for alphabets, values in classes.items():
    if values == predictions[0] :
        print("Possible Alphabet according to the input : ", alphabets)